In [1]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention,Embedding,Dense,Activation,Flatten,Input,BatchNormalization,Dropout,GlobalAveragePooling1D,Concatenate,MultiHeadAttention,LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.metrics import F1Score,Recall,Precision,BinaryAccuracy
from pyvi import ViTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sys
import pandas as pd 
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import max_norm
import tensorflow.keras.backend as K
from sklearn.utils import shuffle
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback

In [ ]:
df = pd.read_csv("synthetic_data_0.txt", delimiter="_", header=None)
data_new = pd.read_csv("synthetic_data_2.txt", delimiter="_", header=None)

In [96]:
print(df.head(1))

           0   1   2           3   4       5   6   \
0  Thiều Việt  18   6  Kỹ sư điện   1  337.41  16   

                                        7   8   9   ...        20       21  \
0  váy maxi, quần yếm nữ, mũ lưỡi trai nam  16   9  ...  Áo khoác  Mùa thu   

              22       23          24  25   26     27  \
0  Hồng cánh sen  Brand D  Streetwear   L  308  79.63   

                                                  28 29  
0  công nghệ tiên tiến bảo hành 12 tháng tư vấn t...  0  

[1 rows x 30 columns]


In [62]:
def create_lookup(data):
    lookup_ = tf.keras.layers.StringLookup(output_mode="int")
    lookup_.adapt(data)
    return lookup_
def encryption(key_word_, data_,lookup_):
    """
        Tokenize theo từ khóa số từ tương ứng với số khóa
    """
    new_data_ = np.array(lookup_(data_)).reshape(-1,1)
    print(f"-{key_word_} trước khi mã hóa: {np.array(data_[0:3]).reshape(3)}   {key_word_} sau khi mã hóa: {new_data_[0:3].reshape(3)}")
    return new_data_

lookup1_ = create_lookup(df.iloc[:,20])
lookup2_ = create_lookup(df.iloc[:,21])
lookup3_ = create_lookup(df.iloc[:,22])
lookup4_ = create_lookup(df.iloc[:,23])
lookup5_ = create_lookup(df.iloc[:,24])
lookup6_ = create_lookup(df.iloc[:,25])
lookup7_ = create_lookup(df.iloc[:,3])

In [ ]:
def create_tokenizer(data_):
    """
        Tạo tokenizer cho dữ liệu đầu vào
    """
    tokenized_data = [ViTokenizer.tokenize(i) for i in data_]
    tokenize_ = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n', split=" ")
    tokenize_.fit_on_texts(tokenized_data)
    return tokenize_
def tokennize(key_word_,data_,tokenize_,_pad_size):
    """
        Tokenize dữ liệu đầu vào
    """
    new_data_ = tokenize_.texts_to_sequences(data_)
    new_data_ = np.array(pad_sequences(new_data_,maxlen=_pad_size,padding="post",truncating="post"))
    print(f"-{key_word_} trước khi tokennize: {data_[0]}    {key_word_} sau khi tokennize: {new_data_[0]}")
    return new_data_

tokennize1_ = create_tokenizer(df.iloc[:,7])
tokennize2_ = create_tokenizer(df.iloc[:,11])
tokennize3_ = create_tokenizer(df.iloc[:,17])
tokennize4_ = create_tokenizer(df.iloc[:,19])
tokennize5_ = create_tokenizer(df.iloc[:,28])

In [68]:
def convert_number_log(key_word_,data_):
    prite_product_ = np.log(data_.astype(np.float64)+1).reshape(-1,1)
    print(f"-{key_word_} trước khi mã hóa: {data_[0]}    {key_word_} sau khi mã hóa: {prite_product_[0]}")
    return prite_product_

In [108]:
age_ = np.array(data_new.iloc[:,1]).astype(np.float64).reshape(-1,1)
print(f"-Tuổi của người dùng: {age_[0:5].reshape(-1)}")
month_user_ = np.array(data_new.iloc[:,2]).astype(np.float64).reshape(-1,1)
print(f"-Tháng sinh của người dùng: {month_user_[0:5].reshape(-1)}")
sex_ = np.array(data_new.iloc[:,4]+1).astype(np.float64).reshape(-1,1)
print(f"-Giới tính của người dùng: {sex_[0:5].reshape(-1)}")
total_money_buy_product_ = convert_number_log(key_word_="Trung bình số tiền mỗi lần mua hàng 30 ngày gần nhất",data_=np.array(data_new.iloc[:,5]))
number_click_ = np.array(data_new.iloc[:,6]+1).astype(np.float64).reshape(-1,1)
print(f"-Số lần click vào sản phẩm: {number_click_[0:5].reshape(-1)}")
number_buy_ = np.array(data_new.iloc[:,8]+1).astype(np.float64).reshape(-1,1)
print(f"-Số lần mua: {number_buy_[0:5].reshape(-1)}")
month_buy_product_ = np.array(data_new.iloc[:,9]).astype(np.float64).reshape(-1,1)
print(f"-Tháng mua sản phẩm: {month_buy_product_[0:5].reshape(-1)}")
total_buy_product_ = np.array(data_new.iloc[:,10]).astype(np.float64).reshape(-1,1)
print(f"-Tổng số lượng sản phẩm mà người dùng mua trong 30 ngày gần nhất: {total_buy_product_[0:5].reshape(-1)}")
number_last_day_buy_product_ = np.array(data_new.iloc[:,12]).astype(np.float64).reshape(-1,1)
print(f"-Khoản cách mua sản phẩm này gần nhất: {number_last_day_buy_product_[0:5].reshape(-1)}")
number_cancel_product_ = np.array(data_new.iloc[:,13]+1).astype(np.float64).reshape(-1,1)
print(f"-Số lần hủy sản phẩm: {number_cancel_product_[0:5].reshape(-1)}")
number_return_product_ = np.array(data_new.iloc[:,14]+1).astype(np.float64).reshape(-1,1)
print(f"-Số lần trả sản phẩm: {number_return_product_[0:5].reshape(-1)}")
number_not_buy_product_ = np.array(data_new.iloc[:,15]+1).astype(np.float64).reshape(-1,1)
print(f"-Số lần thêm nhưng không mua sản phẩm: {number_not_buy_product_[0:5].reshape(-1)}")
rate_product_ = np.array(data_new.iloc[:,16]+1).astype(np.float64).reshape(-1,1)
print(f"-Số điểm sản phẩm: {rate_product_[0:5].reshape(-1)}")
discount_ = np.array(data_new.iloc[:,18]+1).astype(np.float64).reshape(-1,1)
print(f"-Giảm giá: {discount_[0:5].reshape(-1)}")
sales_product_ = np.array(data_new.iloc[:,26]+1).astype(np.float64).reshape(-1,1)
print(f"-Lượt bán sản phẩm trong 30 ngày gần nhất: {sales_product_[0:5].reshape(-1)}")
price_product_ = convert_number_log(key_word_="Giá tiền sản phẩm",data_=np.array(data_new.iloc[:,27]))
search_ = tokennize("3 lịch sử tìm kiếm gần nhất",data_new.iloc[:,7],tokennize1_,12)
name_product_buy_ = tokennize("Tên sản phẩm mua lần gần nhất",data_new.iloc[:,11],tokennize2_,3)
coment_product_ = tokennize("3 Bình luận về sản phẩm",data_new.iloc[:,17],tokennize3_,60)
name_product_ = tokennize("-Tên sản phẩm",data_new.iloc[:,19],tokennize4_,3)
desription_ = tokennize("Mô tả sản phẩm",data_new.iloc[:,28],tokennize5_,131)
type_ = encryption("Loại sản phẩm",data_new.iloc[:,20],lookup1_)
seasion_ = encryption("Mùa ra mắt sản phẩm",data_new.iloc[:,21],lookup2_)
color_ = encryption("Màu sản phẩm",data_new.iloc[:,22],lookup3_)
brand_ = encryption("Thương hiệu sản phẩm",data_new.iloc[:,23],lookup4_)
style_ = encryption("Phong cách sản phẩm",data_new.iloc[:,24],lookup5_)
size_ = encryption("Size sản phẩm",data_new.iloc[:,25],lookup6_)
job_ = encryption("Nghề nghiệp của người dùng",data_new.iloc[:,3],lookup7_)

-Tuổi của người dùng: [50. 20. 34. 54. 57.]
-Tháng sinh của người dùng: [ 2. 12.  9.  4. 11.]
-Giới tính của người dùng: [2. 1. 2. 1. 2.]
-Trung bình số tiền mỗi lần mua hàng 30 ngày gần nhất trước khi mã hóa: 483.91    Trung bình số tiền mỗi lần mua hàng 30 ngày gần nhất sau khi mã hóa: [6.18396331]
-Số lần click vào sản phẩm: [84. 98. 57. 70. 53.]
-Số lần mua: [22. 10. 10. 29. 14.]
-Tháng mua sản phẩm: [6. 6. 2. 2. 6.]
-Tổng số lượng sản phẩm mà người dùng mua trong 30 ngày gần nhất: [18. 48.  9. 26. 14.]
-Khoản cách mua sản phẩm này gần nhất: [ 3. 59.  5.  5.  5.]
-Số lần hủy sản phẩm: [4. 8. 7. 6. 7.]
-Số lần trả sản phẩm: [10.  8.  5.  3.  7.]
-Số lần thêm nhưng không mua sản phẩm: [ 2. 10. 16.  9. 19.]
-Số điểm sản phẩm: [7.8 3.2 7.1 2.5 5.5]
-Giảm giá: [1. 2. 1. 2. 1.]
-Lượt bán sản phẩm trong 30 ngày gần nhất: [312. 241. 683. 373. 313.]
-Giá tiền sản phẩm trước khi mã hóa: 87.38    Giá tiền sản phẩm sau khi mã hóa: [4.4816457]
-3 lịch sử tìm kiếm gần nhất trước khi tokennize: g

In [109]:
data_x1_ = np.concatenate((age_,
                           month_user_,
                           job_,
                           sex_,),axis=1)
data_x2_ = np.concatenate((
                          total_money_buy_product_,
                          number_click_,
                          search_,
                          number_buy_,
                          number_last_day_buy_product_,
                          number_cancel_product_,
                          number_not_buy_product_,
                          number_return_product_
                          ),axis=1)
data_x3_ = np.concatenate((
                          name_product_,
                          type_,
                          seasion_,
                          color_,
                          brand_,
                          style_,
                          size_),axis=1)
data_x4_ = np.concatenate((
                          rate_product_,
                          coment_product_),axis=1)
data_x5_ = np.concatenate((
                          total_buy_product_,
                          price_product_,
                          discount_),axis=1)
data_x6_ = np.array(desription_)
data_x7_ = np.concatenate((        
                          month_buy_product_,
                          name_product_buy_,
                          sales_product_,
                          ),axis=1)

# Kiểm tra lại shape sau khi shuffle
print(f"-Shape dữ liệu nhóm 1: {data_x1_.shape}")
print(f"-Shape dữ liệu nhóm 2: {data_x2_.shape}")
print(f"-Shape dữ liệu nhóm 3: {data_x3_.shape}")
print(f"-Shape dữ liệu nhóm 4: {data_x4_.shape}")
print(f"-Shape dữ liệu nhóm 5: {data_x5_.shape}")
print(f"-Shape dữ liệu nhóm 6: {data_x6_.shape}")
print(f"-Shape dữ liệu nhóm 7: {data_x7_.shape}")

data_y_ = df.iloc[:,-1]

-Shape dữ liệu nhóm 1: (10000, 4)
-Shape dữ liệu nhóm 2: (10000, 19)
-Shape dữ liệu nhóm 3: (10000, 9)
-Shape dữ liệu nhóm 4: (10000, 61)
-Shape dữ liệu nhóm 5: (10000, 3)
-Shape dữ liệu nhóm 6: (10000, 131)
-Shape dữ liệu nhóm 7: (10000, 5)


In [110]:
class PositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, d_model,name_,trainable=True,dtype="float32"):
        super().__init__(name=name_,trainable=trainable,dtype=dtype)
        self.pos_emb = self.add_weight(
            shape=(sequence_length, d_model),  # Ma trận vị trí (số đặc trưng x d_model)
            initializer="random_normal",
            trainable=True,
            name="position_embedding"
        )

    def call(self, inputs, training=True):
        return inputs + self.pos_emb  # Cộng thẳng vào embedding    


class encode(tf.keras.layers.Layer):
    def __init__(self, deep_, num_head_, dropout_rate_, name_,trainable=True,dtype="float32"):
        super(encode, self).__init__(name=name_,trainable=trainable,dtype=dtype)
        self.attention_ = MultiHeadAttention(num_heads = num_head_, key_dim = deep_)
        self.dense_ = Dense(deep_,kernel_regularizer = regularizers.l2(0.001),kernel_constraint=max_norm(3), activation='relu')
        self.l1_ = LayerNormalization(epsilon=1e-6)
        self.dr1_ = Dropout(dropout_rate_)
        self.l2_ = LayerNormalization(epsilon=1e-6)
        self.dr2_ = Dropout(dropout_rate_)

    def call(self, input, training=True):
        x1_ = self.attention_(input[0],input[0])
        # x2_ = self.dr1_(x1_)
        x3_ = self.l1_(x1_+input[1])

        x4_ = self.dense_(x3_)
        # x5_ = self.dr2_(x4_)
        x6_ = self.l2_(x4_+x3_)

        return x6_

class emded(tf.keras.layers.Layer):
    def __init__(self, name_, deep_,trainable=True,dtype="float32"):
        super(emded, self).__init__(name=name_,trainable=trainable,dtype=dtype)
        self.input_dims = [185, 102, 108, 136, 51, 136, 1002]
        self.output_dims = [14, 11, 11, 12, 8, 12, 32]
        self.deep = deep_

        self.embed_layers = [Embedding(input_dim=dim, output_dim=out_dim) 
                             for dim, out_dim in zip(self.input_dims, self.output_dims)]
        self.dense_layers = [Dense(self.deep, use_bias=False) for _ in range(5)]
        self.concat_ = Concatenate(axis=1)

    def call(self, inputs, training=True):
        x1_, x2_, x3_, x4_, x5_,x6_,x7_ = inputs
        x_embedded = [embed(x) for embed, x in zip(self.embed_layers, [x1_, x2_, x3_, x4_, x5_,x6_,x7_])]
        x_dense = [dense(x) for dense, x in zip(self.dense_layers, x_embedded)]
        return self.concat_(x_dense)
        
class output(tf.keras.layers.Layer):
    def __init__(self, dropout_rate, deep_, name_,trainable=True,dtype="float32"):
        super(output, self).__init__(name=name_,trainable=trainable,dtype=dtype)
        self.dense_out1_ = Dense(deep_)
        self.l_out1_ = LayerNormalization(epsilon=1e-6)
        self.d_out1_ = Dropout(0.2)

    def call(self, input, training=True):
        x_ = self.dense_out1_(input)
        x_ = self.l_out1_(x_)
        x_ = self.d_out1_(x_)
        
        return x_

In [111]:
input1_ = Input(shape=(4,))
input2_ = Input(shape=(19,))
input3_ = Input(shape=(9,))
input4_ = Input(shape=(61,))
input5_ = Input(shape=(3,))
input6_ = Input(shape=(131,))
input7_ = Input(shape=(5,))


emded_ = emded("embeding", 256)([input1_,input2_,input3_,input4_,input5_,input6_,input7_])

pos_ = PositionEmbedding(sequence_length=emded_.shape[1], d_model = 256,name_ = "pos_1")(emded_)

encode1_ = encode(deep_ = 256, num_head_ = 2, dropout_rate_ = 0.2, name_="encode_1")([pos_,pos_])

flatten_ = GlobalAveragePooling1D(name="flatten_1")(encode1_)

out1_ = output(dropout_rate = 0.2, deep_ = 256, name_ = "ffn_output_1")(flatten_)

out_full_ = Dense(1,activity_regularizer = regularizers.l2(0.0001),activation='sigmoid',name="output")(out1_)

model_ = Model([input1_,input2_,input3_,input4_,input5_,input6_,input7_],out_full_,name="test_model")
model_.summary()

Model: "test_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_28      │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_29      │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_30      │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_31      │ (None, 61)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_32      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_33      │ (None, 131)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_34      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embeding (emded)    │ (None, 96, 256)   │     54,972 │ input_layer_28[0… │
│                     │                   │            │ input_layer_29[0… │
│                     │                   │            │ input_layer_30[0… │
│                     │                   │            │ input_layer_31[0… │
│                     │                   │            │ input_layer_32[0… │
│                     │                   │            │ input_layer_33[0… │
│                     │                   │            │ input_layer_34[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_1               │ (None, 96, 256)   │     24,576 │ embeding[0][0]    │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encode_1 (encode)   │ (None, 96, 256)   │    592,896 │ pos_1[0][0],      │
│                     │                   │            │ pos_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1           │ (None, 256)       │          0 │ encode_1[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ffn_output_1        │ (None, 256)       │     66,304 │ flatten_1[0][0]   │
│ (output)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        257 │ ffn_output_1[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 739,005 (2.82 MB)

 Trainable params: 739,005 (2.82 MB)

 Non-trainable params: 0 (0.00 B)

In [112]:
threshold_ = 0.6

opt = tf.keras.optimizers.Adam(learning_rate=1e-4,clipnorm=1.0)
loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1)

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", threshold=threshold_, **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.threshold = threshold
        self.true_positives = self.add_weight(name="tp", initializer="zeros")
        self.false_positives = self.add_weight(name="fp", initializer="zeros")
        self.false_negatives = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)  # Chuyển xác suất thành nhãn 0-1
        y_true = tf.cast(y_true, tf.float32)

        tp = tf.reduce_sum(y_true * y_pred)  # True Positives
        fp = tf.reduce_sum((1 - y_true) * y_pred)  # False Positives
        fn = tf.reduce_sum(y_true * (1 - y_pred))  # False Negatives

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)

In [113]:
model_.load_weights("model_weights.weights.h5")
model_.compile(optimizer=opt,loss=loss,metrics=[F1Score(),BinaryAccuracy(),Recall(),Precision()])

In [114]:
h_ = model_.fit(
    [data_x1_,data_x2_,data_x3_,data_x4_,data_x5_,data_x6_,data_x7_],data_y_,
    epochs = 100,
    validation_split = 0.1
) 

Epoch 1/100


d:\appSetting\codeSdk\python\Lib\site-packages\keras\src\optimizers\base_optimizer.py:735: UserWarning: Gradients do not exist for variables ['embeding/embedding_33/embeddings', 'embeding/embedding_34/embeddings'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


 42/282 ━━━━━━━━━━━━━━━━━━━━ 18s 79ms/step - binary_accuracy: 0.9404 - f1_score: 0.9061 - loss: 0.3135 - precision_4: 0.8738 - recall_4: 0.9409

KeyboardInterrupt: 